In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
#Loading Labels
traindf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/traindf.xlsx')
valdf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/valdf.xlsx')
testdf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/testdf.xlsx')
traindf.head()

In [ ]:
traindf['gender'].value_counts()

In [ ]:
#Loading Features Extracted using ResNet50

fx_train_res_pp= np.load('/content/drive/My Drive/Paper/UTKFace/fx_train_res_pp.npy')
fx_val_res_pp = np.load('/content/drive/My Drive/Paper/UTKFace/fx_val_res_pp.npy')
fx_test_res_pp = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_res_pp.npy')
print("Loaded!")

Loaded!


In [ ]:
x_train= np.load('/content/drive/My Drive/Paper/UTKFace/x_train.npy')
x_val = np.load('/content/drive/My Drive/Paper/UTKFace/x_val.npy')
x_test = np.load('/content/drive/My Drive/Paper/UTKFace/x_test.npy')

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git -q

In [ ]:
from keras_vggface.vggface import VGGFace
res = VGGFace(model='resnet50',include_top=False, input_shape=(200, 200, 3))

94699520/94694792 [==============================] - 9s 0us/step


In [ ]:
fx_train_res = res.predict(x_train)
fx_val_res = res.predict(x_val)
fx_test_res = res.predict(x_test)

In [ ]:
np.save('/content/drive/My Drive/Paper/UTKFace/fx_train_res.npy',fx_train_res)
np.save('/content/drive/My Drive/Paper/UTKFace/fx_val_res.npy',fx_val_res)
np.save('/content/drive/My Drive/Paper/UTKFace/fx_test_res.npy',fx_test_res)

Gender Classification Model

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',np.unique(traindf['gender'].values),traindf['gender'].values)
class_weights

In [ ]:
print('Train : '+str(fx_train_res.shape))
print('Val : '+str(fx_val_res.shape))
print('Test : '+str(fx_test_res.shape))

In [ ]:
model = Sequential()

model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128,activation='elu',kernel_initializer='he_uniform',kernel_constraint=tf.keras.constraints.max_norm(3)))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
sgd = tf.keras.optimizers.SGD(
    learning_rate=1e-4, momentum=0, nesterov=False, name="SGD")

adam = tf.keras.optimizers.Adam(
    learning_rate=1e-5,
    amsgrad=False,)

adamax = tf.keras.optimizers.Adamax(learning_rate=5e-5)

model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(fx_train_res,traindf['gender'].values,epochs=2, validation_data=(fx_val_res,valdf['gender'].values),class_weight=class_weights,batch_size=128)

In [ ]:
model.evaluate(fx_test_res,testdf['gender'].values)

In [ ]:
model.save('/content/drive/My Drive/Paper/UTKFace/Gender/ResNet50/model.h5')